## Task:

Susan has created a daily climatology of watershed discharges, based on monthly means. SEE:
https://github.com/SalishSeaCast/tools/blob/master/I_ForcingFiles/Rivers/SeasonalCyclesFromMonthlyMeans.ipynb

Daily files read into the model are currently in the form:
    
    "R201702DFraCElse_y2017m01d10.nc" 
stored in:
    
    /results/forcing/rivers
where the Fraser has a daily real-time value from the Hope gauge, and the rest of the rivers are the morison climatology. 

We need to create files of the above form, one file per day, except replacing the Morrison climatology with the interpolated climatology. Need to be v.v. careful with units, as m3/s units are not used in the model. 

    Rivers are treated like rain, added from above.  So kg of water/m2 /s where the m2 is horizontal area.  This we will call nf (nemo flux)
    So for m3/s first we convert from mass to volume
    nf / rho (fresh water so 1000 kg/m3)  so now we have m/s
    Then multiply by the area of the grid cell
     nf  / rho * e1t(i,j) *e2t(i,j)

The steps are as follows:

    1) Open the river climatology file from 
        rivers-climatology/watershed_climatologies/Salish_allrivers_daily.nc
    2) use the river properties dictionary, the 'prop' key to distribute the climatology among individual rivers, still in m3/s, and save in a new dictionary -
    3) translate from m3/s to kg/m2/s, save that into the new dictionary
             ie eg river "nooksack" will have 365 numbers, in kg/m2/s 
    4) create new daily files and populate them with the numbers from 3, using the i and j values found in the river properties dictionary. These daily files are of a similar type as "R201702DFraCElse_y2017m01d10.nc" 

In [55]:
%reset
import netCDF4 as nc
import numpy as np
import arrow

import sys
sys.path.append('/data/tjarniko/MEOPAR/tools/SalishSeaTools/salishsea_tools/')
#import mocsy
import river_201702 as rv
import river_downbyone2 as rv2

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


### Trying to understand inconsistency between how rivers are seen in the different grids - using downbyone grid for the R201702DFraCElse_y2017m01d10.nc and other files...

In [56]:
print('river is Oyster, properties')
print('rivers201702')
print(rv.prop_dict['evi_n']['Oyster'])
print('riversdownbyone')
print(rv2.prop_dict['evi_n']['Oyster'])
print('note j and i are backwards')

print('output from j = 706, i = 123 from rivers/rivers_month_201702.nc')
rivdat = nc.Dataset('/data/tjarniko/MEOPAR/rivers/rivers_month_201702.nc')
t_runoff = rivdat.variables['rorunoff'][8,706,123]
print(t_runoff)
print('output from j = 706, i = 123 from results/forcing/rivers/RLonFraCElse_y2017m08d07.nc')
sample_river_file = \
nc.Dataset('/results/forcing/rivers/RLonFraCElse_y2017m08d07.nc')
print(sample_river_file['rorunoff'][0,706,123])
print('output from wider area around this point')
print(sample_river_file['rorunoff'][0,706-2:706+2,123-2:123+2])
print('output from j = 705, i = 122 (so off by 1)')
print(sample_river_file['rorunoff'][0,705,122])
print('so, populate using the downbyone grid')

river is Oyster, properties
rivers201702
{'prop': 0.03738799052425584, 'i': 706, 'j': 123, 'di': 1, 'dj': 1, 'depth': 1}
riversdownbyone
{'prop': 0.03738799052425584, 'i': 705, 'j': 122, 'di': 1, 'dj': 1, 'depth': 3}
note j and i are backwards
output from j = 706, i = 123 from rivers/rivers_month_201702.nc
0.033809192
output from j = 706, i = 123 from results/forcing/rivers/RLonFraCElse_y2017m08d07.nc
0.0
output from wider area around this point
[[0.         0.         0.         0.        ]
 [0.         0.03536774 0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]
output from j = 705, i = 122 (so off by 1)
0.035367742
so, populate using the downbyone grid


In [57]:
sample_river_file = \
nc.Dataset('/results/forcing/rivers/RLonFraCElse_y2017m08d07.nc')

#print(sample_river_file)

climatology_file = \
nc.Dataset('/data/tjarniko/MEOPAR/rivers-climatology/watershed_climatologies/Salish_allrivers_daily.nc')
#print(climatology_file['fraser'][300])


### code to get interpolated runoff for every river for every day of the year, in both m3/s and model units (kg/m2/s)

#### if a river has di or dj > 1, print out its properties

#### do i multiply anything for depth?

In [58]:
import copy
clim_dict = copy.deepcopy(rv2.prop_dict)
grid = nc.Dataset('/data/tjarniko/MEOPAR/grid/mesh_mask201702.nc')

watersheds = ['howe', 'jdf', 'puget', 'skagit', 'fraser', \
              'evi_n', 'jervis', 'toba', 'bute', 'evi_s']

for w in watersheds:
    interp_clim = climatology_file[w][:]
    print('')
    print(w)
    #print(interp_clim)
    rivers = (rv2.prop_dict[w].keys())
    for r in rivers:
        #print(r)
        prop_wshed = rv2.prop_dict[w][r]['prop']
        clim_dict[w][r]['interp_clim'] = prop_wshed * interp_clim
        river_i = rv2.prop_dict[w][r]['j']
        river_j = rv2.prop_dict[w][r]['i']
        river_dj = rv2.prop_dict[w][r]['di']
        river_di = rv2.prop_dict[w][r]['dj']
        area = grid['e1t'][0,river_j,river_i]*grid['e2t'][0,river_j,river_i]
        clim_dict[w][r]['interp_clim_modunits'] = prop_wshed * interp_clim / area * 1000
       # if ((river_dj > 1) | (river_di > 1)):
       #     print(r)
       #     print(rv2.prop_dict[w][r])
#runoff_m3 = (year_runoff / 1000) * t_area
#runoff in model units = runoff_m3 / t_area * 1000 

print(rv2.prop_dict['fraser'])


howe

jdf

puget

skagit

fraser

evi_n

jervis

toba

bute

evi_s
{'Dakota': {'prop': 0.00096, 'i': 362, 'j': 357, 'di': 1, 'dj': 1, 'depth': 3}, 'Terrel': {'prop': 0.00064, 'i': 351, 'j': 345, 'di': 1, 'dj': 1, 'depth': 3}, 'Nooksack': {'prop': 0.012, 'i': 321, 'j': 347, 'di': 1, 'dj': 1, 'depth': 3}, 'Squallum': {'prop': 0.0008, 'i': 305, 'j': 365, 'di': 1, 'dj': 1, 'depth': 3}, 'Lakethingo': {'prop': 0.00096, 'i': 302, 'j': 367, 'di': 1, 'dj': 1, 'depth': 3}, 'Chuckanut': {'prop': 0.00064, 'i': 298, 'j': 361, 'di': 1, 'dj': 1, 'depth': 3}, 'Fraser1': {'prop': 0.738, 'i': 500, 'j': 395, 'di': 1, 'dj': 1, 'depth': 3}, 'Fraser2': {'prop': 0.0492, 'i': 409, 'j': 315, 'di': 2, 'dj': 1, 'depth': 3}, 'Fraser3': {'prop': 0.0492, 'i': 434, 'j': 318, 'di': 2, 'dj': 1, 'depth': 3}, 'Fraser4': {'prop': 0.14759999999999998, 'i': 440, 'j': 323, 'di': 1, 'dj': 2, 'depth': 3}}


In [59]:
print(rv2.prop_dict['howe']['Squamish']['i'])
#{'prop': 0.9, 'i': 532, 'j': 385, 'di': 1, 'dj': 2, 'depth': 3}
sample_river_file = \
nc.Dataset('/results/forcing/rivers/RLonFraCElse_y2017m08d07.nc')

print(sample_river_file['rorunoff'][0,532-2:532+2,385-2:385+2])

print(clim_dict['howe']['Squamish']['interp_clim_modunits'][219])


print(rv2.prop_dict['evi_n']['Oyster'])
print(sample_river_file['rorunoff'][0,706-1:706+1,123-1:123+1])
print(clim_dict['evi_n']['Oyster']['interp_clim_modunits'][218])

532
[[0.        0.        0.        0.       ]
 [0.        0.        0.        0.       ]
 [0.        0.        1.5795676 1.5795676]
 [0.        0.        0.        0.       ]]
3.244538
{'prop': 0.03738799052425584, 'i': 705, 'j': 122, 'di': 1, 'dj': 1, 'depth': 3}
[[0.03536774 0.        ]
 [0.         0.        ]]
0.03558299


#### checking the Squamish river interpolated result against a daily file

In [60]:
print(rv2.prop_dict['howe'])


riv_i = clim_dict['howe']['Squamish']['i']
riv_j = clim_dict['howe']['Squamish']['j']
riv_di = clim_dict['howe']['Squamish']['di']
riv_dj = clim_dict['howe']['Squamish']['dj']
print(riv_i)
#print(sample_river_file['rorunoff'][0,532-2:532+2,385-2:385+2])
print(sample_river_file['rorunoff'][0,riv_i:riv_i+riv_di,riv_j:riv_j+riv_dj])
print(clim_dict['howe']['Squamish']['interp_clim_modunits'][219])

print(rivdat.variables['rorunoff'][8,534,384:386])


{'Squamish': {'prop': 0.9, 'i': 532, 'j': 385, 'di': 1, 'dj': 2, 'depth': 3}, 'Burrard': {'prop': 0.1, 'i': 457, 'j': 343, 'di': 3, 'dj': 1, 'depth': 3}}
532
[[1.5795676 1.5795676]]
3.244538
[1.0668372 1.0668372]


#### checking the homathko river interpolated result against a daily file

In [61]:
print(rv2.prop_dict['bute'])


riv_i = clim_dict['bute']['Homathko']['i']
riv_j = clim_dict['bute']['Homathko']['j']
riv_di = clim_dict['bute']['Homathko']['di']
riv_dj = clim_dict['bute']['Homathko']['dj']
print(riv_i)
#print(sample_river_file['rorunoff'][0,532-2:532+2,385-2:385+2])
print(sample_river_file['rorunoff'][0,riv_i:riv_i+riv_di,riv_j:riv_j+riv_dj])
print(clim_dict['bute']['Homathko']['interp_clim_modunits'][219])



{'Homathko': {'prop': 0.58, 'i': 897, 'j': 294, 'di': 1, 'dj': 1, 'depth': 3}, 'Southgate': {'prop': 0.35, 'i': 885, 'j': 296, 'di': 1, 'dj': 2, 'depth': 3}, 'Orford': {'prop': 0.07, 'i': 831, 'j': 249, 'di': 1, 'dj': 1, 'depth': 3}}
897
[[3.4198332]]
3.5753164


### 

In [62]:
print(np.shape(sample_river_file['rorunoff'][0,:,:]))
print(np.max((sample_river_file['rorunoff'][0,:,:])))

for w in watersheds:
    interp_clim = climatology_file[w][:]
    print('')
    print(w)
    #print(interp_clim)
    rivers = (rv2.prop_dict[w].keys())
    for r in rivers:
        #print(r)
        prop_wshed = rv2.prop_dict[w][r]['prop']
        clim_dict[w][r]['interp_clim'] = prop_wshed * interp_clim
        river_i = rv2.prop_dict[w][r]['j']

        river_j = rv2.prop_dict[w][r]['i']
        river_di = rv2.prop_dict[w][r]['di']
        river_dj = rv2.prop_dict[w][r]['dj']
#         print(river_i)
#         print(river_j)
        print(r)
        print(sample_river_file['rorunoff'][0,river_j,river_i])
        print('')
        area = grid['e1t'][0,river_j,river_i]*grid['e2t'][0,river_j,river_i]
        clim_dict[w][r]['interp_clim_modunits'] = prop_wshed * interp_clim / area * 1000

(898, 398)
12.633769

howe
Squamish
1.5795676

Burrard
0.1150185


jdf
SanJuan
0.034642108

Gordon
0.014698593

Loss
0.00523818

Jordan
0.0052331225

Muir
0.0052300794

Tugwell
0.005229909

Sooke
0.034496814

Elwha
0.031203095

Tumwater
0.0006239175

Valley
0.00062389136

Ennis
0.0012476821

Morse
0.0043668696

Bagley
0.0012476125

Siebert
0.0012475995

McDonald
0.0018716713

DungenessMatriotti
0.019972652

Coville
0.0020797825

Salt
0.0020803572

Field
0.0020789178

Lyre
0.00831586

EastWestTwin
0.0020778405

Deep
0.002077297

Pysht
0.004156416

Clallom
0.004158345

Hoko
0.008319551

Sekiu
0.004159485

Sail
0.002083185


puget
Johnson
0.0074257967

Jimmycomelately
0.0074195843

SalmonSnow
0.03703793

Chimacum
0.029645022

Thorndike
0.0073715327

Torboo
0.0073816506

LittleBigQuilcene
0.05166161

Dosewalips
0.029476227

Duckabush
0.020632207

Fulton
0.0029481018

Waketick
0.0029488364

HammaHamma
0.02064309

Jorsted
0.0029493005

Eagle
0.002950191

Lilliwaup
0.0029519103

Finch
0.00295

In [63]:
start ='2017-01-01'
end ='2017-01-03'

start_run = arrow.get(start)
end_run = arrow.get(end)

arrow_array = []

for r in arrow.Arrow.span_range('day', start_run, end_run):
    arrow_array.append(r)


for i in range(0,3):

    tdate = arrow_array[i][0]
    y = tdate.format('YYYY')
    m = tdate.format('MM')
    d = tdate.format('DD')

    tday = f"y{y}m{m}d{d}"
    print(tday)

y2017m01d01
y2017m01d02
y2017m01d03


In [64]:
grid = nc.Dataset('/data/tjarniko/MEOPAR/grid/mesh_mask201702.nc')
print(grid['e1t'][0,250,250])
print(np.shape(grid['e1t']))

440.54666691637465
(1, 898, 398)
